In [1]:
import sys
from transformers import AutoTokenizer

# ================= 修改配置 =================
#TEACHER_MODEL_PATH = os.path.join(BASE_DIR, "teacher_7b")
#STUDENT_MODEL_PATH = os.path.join(BASE_DIR, "student_0.5b")
#TEACHER_MODEL_PATH = "Qwen/Qwen3-4B-Instruct-2507"
TEACHER_MODEL_PATH = "Qwen/Qwen2.5-7B-Instruct"
STUDENT_MODEL_PATH = "Qwen/Qwen2.5-0.5B-Instruct"

print(f"正在从本地加载老师模型: {TEACHER_MODEL_PATH}")
print(f"正在从本地加载学生模型: {STUDENT_MODEL_PATH}")

# Tokenizer 加载 (直接读本地文件夹)
tokenizer_s = AutoTokenizer.from_pretrained(STUDENT_MODEL_PATH, trust_remote_code=True)
tokenizer_t = AutoTokenizer.from_pretrained(TEACHER_MODEL_PATH, trust_remote_code=True)

# 确保 pad_token 存在 (Qwen 有时默认 pad_token 为 None)
if tokenizer_s.pad_token is None: tokenizer_s.pad_token = tokenizer_s.eos_token
if tokenizer_t.pad_token is None: tokenizer_t.pad_token = tokenizer_t.eos_token

# 核心检查：白盒蒸馏要求老师和学生的 Logits 对应同一个词表
if tokenizer_s.vocab_size != tokenizer_t.vocab_size:
    print(f"[错误] 词表不匹配！学生: {tokenizer_s.vocab_size}, 老师: {tokenizer_t.vocab_size}")
    print("Logits 无法对齐，无法进行白盒蒸馏。程序退出。")
    sys.exit(1)
else:
    print(">>> 验证通过：老师和学生词表一致。")
    tokenizer = tokenizer_s # 后续统一使用一个 tokenizer

正在从本地加载老师模型: Qwen/Qwen2.5-7B-Instruct
正在从本地加载学生模型: Qwen/Qwen2.5-0.5B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

>>> 验证通过：老师和学生词表一致。


In [4]:
import os
from huggingface_hub import snapshot_download

# ================= 配置下载路径 =================
# 建议在当前目录下创建一个 models 文件夹，方便管理
BASE_DIR = "./local_models"

# 1. 老师模型 ID (假设是 GPTQ 量化版)
TEACHER_ID = "Qwen/Qwen2.5-7B-Instruct"
#TEACHER_ID = "Qwen/Qwen3-4B-Instruct-2507"
# 老师模型本地存储路径
TEACHER_DIR = os.path.join(BASE_DIR, "teacher_3_4b")

# 2. 学生模型 ID
STUDENT_ID = "Qwen/Qwen2.5-0.5B-Instruct"
# 学生模型本地存储路径
STUDENT_DIR = os.path.join(BASE_DIR, "student_0.5b")

# ================= 开始下载 =================
print(f"准备下载模型到: {BASE_DIR}")

# 下载老师模型
if not os.path.exists(TEACHER_DIR):
    print(f"正在下载老师模型: {TEACHER_ID} ...")
    snapshot_download(
        repo_id=TEACHER_ID,
        local_dir=TEACHER_DIR,
        local_dir_use_symlinks=False, # 设为 False 确保下载的是实际文件而不是快捷方式
        resume_download=True          # 支持断点续传
    )
    print("老师模型下载完成！")
else:
    print(f"老师模型目录已存在，跳过下载: {TEACHER_DIR}")

# 下载学生模型
if not os.path.exists(STUDENT_DIR):
    print(f"正在下载学生模型: {STUDENT_ID} ...")
    snapshot_download(
        repo_id=STUDENT_ID,
        local_dir=STUDENT_DIR,
        local_dir_use_symlinks=False,
        resume_download=True
    )
    print("学生模型下载完成！")
else:
    print(f"学生模型目录已存在，跳过下载: {STUDENT_DIR}")

print("\n所有模型准备就绪！")
print(f"老师模型路径: {os.path.abspath(TEACHER_DIR)}")
print(f"学生模型路径: {os.path.abspath(STUDENT_DIR)}")

准备下载模型到: ./local_models
老师模型目录已存在，跳过下载: ./local_models/teacher_3_4b
学生模型目录已存在，跳过下载: ./local_models/student_0.5b

所有模型准备就绪！
老师模型路径: /Users/lhc456/Desktop/python/play-with-instruct-model/cot/local_models/teacher_3_4b
学生模型路径: /Users/lhc456/Desktop/python/play-with-instruct-model/cot/local_models/student_0.5b


In [6]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 17.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.2
    Uninstalling huggingface-hub-0.25.2:
      Successfully uninstalled huggingface-hub-0.25.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.1
    Uninstalling tokenizers-0.20.1:
      Successfully uninstalled tokenizers-0.20.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2


In [8]:
import torch
import torch.nn.functional as F
import sys
import os
from datasets import load_dataset
from transformers import (
    Trainer, 
    TrainingArguments, 
    AutoModelForCausalLM, 
    AutoTokenizer
)
from trl import DataCollatorForCompletionOnlyLM

# ==============================================================================
# 1. 配置参数
# ==============================================================================
BASE_DIR = "./local_models"

TEACHER_MODEL_PATH = os.path.join(BASE_DIR, "teacher_7b")
STUDENT_MODEL_PATH = os.path.join(BASE_DIR, "student_0.5b")
#TEACHER_MODEL_PATH = "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4" 
#STUDENT_MODEL_PATH = "Qwen/Qwen2.5-0.5B-Instruct"
DATA_FILE = "math_dataset_with_answers_new.jsonl" 

# 训练超参数
MAX_LENGTH = 1024  

# TEMPERATURE (温度 T)的作用：
# 原始情况：输出的 Logits 可能是 [猫: 10, 狗: 2, 汽车: -5]。
# 经过 Softmax 后，变成 [0.999, 0.001, 0.0]。这样“狗”和“汽车”的区别被抹平了
# 但白盒蒸馏，就是想把老师模型中，这些差别学到。
# 所以把 Logits 除以一个温度（比如 T=2.0）。数值变成 [5, 1, -2.5]。Softmax 后可能变成 [0.8, 0.15, 0.05]。
# 效果：概率分布变“平缓”了。 这样“狗(0.15)”和“汽车(0.05)”的区别就显现出来了，学生就能学到这些细节。
TEMPERATURE = 2.0  
 
# ALPHA (比如 0.5)：
#   0.5 份的努力：去对齐标准答案（做对题）。
#   0.5 份的努力：去模仿老师的思考方式（学气质）。
#  Loss Total=α⋅Loss_CE+(1−α)⋅Loss_KD
ALPHA = 0.5 

BATCH_SIZE = 2           
GRAD_ACCUMULATION = 4    
EPOCHS = 3               

# ==============================================================================
# 2. 加载与校验 Tokenizer
# ==============================================================================
print(">>> 正在校验 Tokenizer...")
# 这里的 trust_remote_code=True 是必须的，防止某些自定义模型报错
tokenizer_s = AutoTokenizer.from_pretrained(STUDENT_MODEL_PATH, trust_remote_code=True)
tokenizer_t = AutoTokenizer.from_pretrained(TEACHER_MODEL_PATH, trust_remote_code=True)

if tokenizer_s.pad_token is None: tokenizer_s.pad_token = tokenizer_s.eos_token
if tokenizer_t.pad_token is None: tokenizer_t.pad_token = tokenizer_t.eos_token

# 校验逻辑
if tokenizer_s.vocab_size != tokenizer_t.vocab_size:
    print(f"[错误] 词表不匹配！学生: {tokenizer_s.vocab_size}, 老师: {tokenizer_t.vocab_size}")
    sys.exit(1)
else:
    print(f">>> 验证通过：逻辑词表大小一致 ({tokenizer_s.vocab_size})。")
    tokenizer = tokenizer_s 

# ==============================================================================
# 3. 数据处理
# ==============================================================================
print(f">>> 正在读取数据: {DATA_FILE}")
dataset = load_dataset("json", data_files=DATA_FILE, split="train")

def process_func(example):
    # 1. 拼接文本
    combined_content = f"[思考过程]\n{example['thinking']}\n\n[最终答案]\n{example['output']}"
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": example['instruction'] + ("\n" + example['input'] if example.get('input') else "")},
        {"role": "assistant", "content": combined_content}
    ]
    # 2. 转字符串
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    # 3. 转数字
    tokenized = tokenizer(text, max_length=MAX_LENGTH, truncation=True)
    return tokenized

print(">>> 处理数据中 (Tokenizing)...")
tokenized_dataset = dataset.map(process_func, remove_columns=dataset.column_names)

# ==============================================================================
# 4. Collator 设置
# ==============================================================================
response_template = "<|im_start|>assistant\n" 
collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
    mlm=False 
)

# ==============================================================================
# 5. 加载模型 (核心修改部分)
# ==============================================================================
print(">>> 加载 Teacher (Int4)...")
# 老师不训练，可以是 FP16 甚至 Int4，没问题
teacher_model = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL_PATH,
    device_map="cuda",
    torch_dtype=torch.float16, 
    trust_remote_code=True
)
teacher_model.eval()
for param in teacher_model.parameters(): param.requires_grad = False

print(">>> 加载 Student (FP32)...")
# 【核心修复！！！】
# 学生模型必须用 float32 加载，或者不写 torch_dtype (默认就是 float32)
# 这样 Trainer(fp16=True) 才能正常建立混合精度的 Master Weights
student_model = AutoModelForCausalLM.from_pretrained(
    STUDENT_MODEL_PATH,
    device_map="cuda",
    torch_dtype=torch.float32,  # <--- 改为 float32，解决 unscale 报错
    trust_remote_code=True
)

# ==============================================================================
# 6. 自定义 Trainer (带维度对齐)
# ==============================================================================
class KDTrainer(Trainer):
    def __init__(self, teacher_model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # 1. 学生前向
        student_outputs = model(**inputs)
        student_logits = student_outputs.logits
        
        # 2. 老师前向
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits
        
        # 3. 【维度对齐】解决 152064 vs 151936 报错
        if student_logits.shape[-1] != teacher_logits.shape[-1]:
            min_dim = min(student_logits.shape[-1], teacher_logits.shape[-1])
            student_logits = student_logits[..., :min_dim]
            teacher_logits = teacher_logits[..., :min_dim]
        
        # 4. 计算 KD Loss

        # KL 散度：这是用来衡量两个概率分布（学生预测的分布 vs 老师预测的分布）有多大差异的数学工具。
        # 目标：让学生的概率分布尽可能趋近于老师的分布。
        loss_fct = torch.nn.KLDivLoss(reduction="batchmean")

        # 问题1：为什么学生是 log_softmax 而老师是 softmax？
        # 这是 PyTorch nn.KLDivLoss 接口的特殊要求：
        #   输入（Input）：要求是 Log 概率（所以学生用 log_softmax）。
        #   目标（Target）：要求是普通概率（所以老师用 softmax）。
        
        # 问题2：为什么要乘以 (TEMPERATURE ** 2)？
        # 这是一个梯度补偿机制。当你把 Logits 除以T时，反向传播计算出来的梯度数值会缩小 1/T^2 倍。
        # 为了让 KD Loss 的梯度大小和 CE Loss 在同一个数量级，不让它变得太小而被忽略，所以人为乘回T^2
        loss_kd = loss_fct(
            F.log_softmax(student_logits / TEMPERATURE, dim=-1),
            F.softmax(teacher_logits / TEMPERATURE, dim=-1)
        ) * (TEMPERATURE ** 2)
        
        # CE Loss让学生学会：这道题答案是 "12"。
        # KD Loss让学生学会：为什么老师在算出 "12" 之前，对 "10" 或 "15" 也有过一点点概率的侧重（可能是中间步骤的近似）。
        loss_ce = student_outputs.loss #普通的交叉熵损失，即答案的损失
        total_loss = ALPHA * loss_ce + (1 - ALPHA) * loss_kd
        
        return (total_loss, student_outputs) if return_outputs else total_loss

# ==============================================================================
# 7. 训练
# ==============================================================================
#所谓混合精度就是计算时用高精度，存放到显存中的用低精度
training_args = TrainingArguments(
    output_dir="./qwen_math_distilled",
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION,
    learning_rate=2e-5,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,                # 开启混合精度，此时需要 Student 是 FP32， 省显存、提速度、防溢出。
    report_to="none",
    remove_unused_columns=True
)

trainer = KDTrainer(
    teacher_model=teacher_model,
    model=student_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=collator,
    tokenizer=tokenizer
)

print(">>> 开始训练...")
torch.cuda.empty_cache()
trainer.train()

print(">>> 保存模型...")
trainer.save_model("./final_math_student_model")
print("完成！")

ImportError: cannot import name 'DataCollatorForCompletionOnlyLM' from 'trl' (/Users/lhc456/opt/anaconda3/envs/llma3/lib/python3.9/site-packages/trl/__init__.py)

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ================= 配置 =================
# 指向刚才保存的微调后的模型目录
MODEL_PATH = "./final_math_student_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"正在加载模型: {MODEL_PATH} ...")

# 1. 加载模型和分词器
# 注意：这里加载的是你训练好的 Student，不需要再加载 Teacher 了
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map=device,
    torch_dtype=torch.float16
)
model.eval() # 切换到评估模式

# 2. 定义推理函数
def generate_answer(instruction, input_text=""):
    # 构造与训练时完全一致的 Prompt 格式
    # 注意：训练时我们把 input 拼在了 instruction 后面
    user_content = instruction + ("\n" + input_text if input_text else "")
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_content}
    ]
    
    # 应用聊天模板 (只生成到 user 结束，后面让模型续写 assistant)
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True # 这一步会自动加上 <|im_start|>assistant\n
    )
    
    # 转为数字
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    # 开始生成
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,      # 允许生成的最大长度
            do_sample=False,          # 采样模式，稍微有点随机性，更像人说话
            temperature=0.2,         # 控制创造性
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # 解码 (只看生成的 Assistant 部分)
    # outputs[0] 包含了输入的 prompt + 生成的 response，我们需要切片
    input_len = inputs.input_ids.shape[1]
    generated_ids = outputs[0][input_len:]
    final_output = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    return final_output

# ================= 测试案例 =================
print("\n>>> 开始测试 (请观察是否包含 [思考过程])\n")

# 测试题 1：训练集中类似的题目（看过拟合/记忆能力）
q1 = "计算：24.6 ÷ 0.6"
print(f"问题: {q1}")
print("-" * 30)
ans1 = generate_answer(q1)
print(ans1)
print("=" * 50)

# 测试题 2：一道新题目（看泛化能力）
q2 = "小明有 5 个苹果，小红的苹果数量是小明的 3 倍少 2 个。请问小红有多少个苹果？"
print(f"\n问题: {q2}")
print("-" * 30)
ans2 = generate_answer(q2)
print(ans2)
print("=" * 50)

# 测试题 3：更难一点的逻辑
q3 = "如果 3x + 5 = 20，那么 x 等于多少？"
print(f"\n问题: {q3}")
print("-" * 30)
ans3 = generate_answer(q3)
print(ans3)
print("=" * 50)

正在加载模型: ./final_math_student_model ...


OSError: Incorrect path_or_model_id: './final_math_student_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

# ================= 配置 =================
# 原始基座模型 ID
BASE_MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"
# 训练好的模型路径
DISTILLED_MODEL_PATH = "./final_math_student_model"

device = "cuda" if torch.cuda.is_available() else "cpu"

print(">>> 正在初始化环境，准备加载两个模型进行 PK...")

# ================= 1. 加载分词器 =================
# 两个模型用的分词器是一样的，加载其中一个即可
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# ================= 2. 加载 原始基座模型 (Old) =================
print(f"Loading Base Model: {BASE_MODEL_ID} ...")
model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map=device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True # 加速加载，减少内存占用
)
model_base.eval()

# ================= 3. 加载 蒸馏后模型 (New) =================
print(f"Loading Distilled Model: {DISTILLED_MODEL_PATH} ...")
model_distilled = AutoModelForCausalLM.from_pretrained(
    DISTILLED_MODEL_PATH,
    device_map=device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model_distilled.eval()

print("\n>>> 模型加载完毕！开始对比测试...\n")

# ================= 4. 定义通用的推理函数 =================
def get_response(model, question):
    """
    输入模型和问题，返回回答。
    使用贪婪搜索 (do_sample=False)，确保输出模型认为概率最大的答案，减少随机性。
    """
    # 构造 Prompt
    # 注意：我们使用标准的 User/Assistant 格式
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]
    
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False, # 【重要】关闭采样，看模型的真实实力，不让它瞎猜
            temperature=None, # 关闭采样后，temperature 无效，设为 None
            top_p=None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
    # 解码
    input_len = inputs.input_ids.shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return response

# ================= 5. 测试案例与对比循环 =================

questions = [
    # 1. 训练集类似的题目（看是否学会了格式）
    "计算：24.6 ÷ 0.6",
    
    # 2. 简单的应用题（看逻辑是否变强）
    "小明有 5 个苹果，小红的苹果数量是小明的 3 倍少 2 个。请问小红有多少个苹果？",
    
    # 3. 逻辑推理题（看是否能分步思考）
    "如果 3x + 5 = 20，那么 x 等于多少？",
    
    # 4. 常识/非数学题（看是否灾难性遗忘，即学了数学忘了怎么说话）
    "你好，请介绍一下你自己。"
]

separator = "=" * 80

for i, q in enumerate(questions):
    print(f"\n📝 测试题目 {i+1}: {q}")
    print(separator)
    
    # --- 原始模型回答 ---
    print("🔴 [原始 0.5B 模型] 回答：")
    try:
        ans_base = get_response(model_base, q)
        print(ans_base.strip())
    except Exception as e:
        print(f"Error: {e}")

    print("-" * 40) # 分割线
    
    # --- 蒸馏模型回答 ---
    print("🟢 [蒸馏后 0.5B 模型] 回答：")
    try:
        ans_distilled = get_response(model_distilled, q)
        print(ans_distilled.strip())
    except Exception as e:
        print(f"Error: {e}")
        
    print(separator)
    # 稍微停顿一下，防止刷新太快
    time.sleep(0.5)

>>> 正在初始化环境，准备加载两个模型进行 PK...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0b0feb24-58fa-41b1-8a90-292cb30dfe59)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Loading Base Model: Qwen/Qwen2.5-0.5B-Instruct ...
Loading Distilled Model: ./final_math_student_model ...


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



>>> 模型加载完毕！开始对比测试...


📝 测试题目 1: 计算：24.6 ÷ 0.6
🔴 [原始 0.5B 模型] 回答：
要计算 \(24.6 \div 0.6\)，我们可以将除法转换为乘法。具体步骤如下：

\[24.6 \div 0.6 = 24.6 \times \frac{1}{0.6}\]

由于 \(0.6 = \frac{3}{5}\)，我们有：

\[24.6 \times \frac{1}{\frac{3}{5}} = 24.6 \times \frac{5}{3}\]

现在，我们将分数相乘：

\[= 8.2 \times 5\]

\[= 41\]

因此，\(24.6 \div 0.6 = 41\)。
----------------------------------------
🟢 [蒸馏后 0.5B 模型] 回答：
要计算24.6除以0.6，可以将这个除法问题简化为两个数相除的问题。具体步骤如下：

1. 将24.6和0.6都转换成相同的小数位数来方便计算。
2. 计算24.6除以0.6。

首先，我们可以直接进行计算，也可以先将24.6和0.6转换为更简单的形式，以便于计算。但在这里，我们直接使用24.6除以0.6来进行计算。

\[24.6 ÷ 0.6 = 41\]

因此，24.6除以0.6的结果是41。

最终答案是41。

📝 测试题目 2: 小明有 5 个苹果，小红的苹果数量是小明的 3 倍少 2 个。请问小红有多少个苹果？
🔴 [原始 0.5B 模型] 回答：
小红的数量是小明的 3 倍少 2 个，所以我们可以这样计算：

小红的数量 = 小明的数量 - (小明的数量的 3 倍) + 2

将小明的数量代入公式中得到：
小红的数量 = 5 - (5 * 3) + 2
小红的数量 = 5 - 15 + 2
小红的数量 = -10 + 2
小红的数量 = -8

但是这个结果显然是不可能的，因为数量不能为负数。这可能是因为题目中的条件没有完全满足，或者我们对问题的理解存在偏差。

如果问题是问的是小红实际拥有的苹果数量，那么答案应该是小红比小明多出的苹果数量加上小明原有的苹果数量。即：
- 小红的数量 = 小明的数量 + (小明的数量的 3 倍)
- 小红的数量 = 5 + (5 * 3)
- 小红的数量 = 5

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

# ================= 配置 =================
BASE_MODEL_ID = "local_models/student_0.5b/"
DISTILLED_MODEL_PATH = "./final_math_student_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(">>> 正在初始化环境...")

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

print(f"Loading Base Model...")
model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID, device_map=device, torch_dtype=torch.float16, low_cpu_mem_usage=True
)
model_base.eval()

print(f"Loading Distilled Model...")
model_distilled = AutoModelForCausalLM.from_pretrained(
    DISTILLED_MODEL_PATH, device_map=device, torch_dtype=torch.float16, low_cpu_mem_usage=True
)
model_distilled.eval()

# ================= 改进后的推理函数 =================
def get_response(model, question, force_thinking=False):
    """
    force_thinking: 如果为 True，强制给模型“喂”一个开头，逼它进入思考模式
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]
    
    # 1. 生成基础 Prompt
    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # 2. 【核心技巧】Prompt 注入
    # 如果是蒸馏模型，我们要检查它是否包含标签。
    # 这里我们手动把 [思考过程] 拼在 Assistant 的开头
    if force_thinking:
        text += "[思考过程]\n"
    
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False, 
            temperature=None,
            top_p=None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
    # 解码
    # 注意：如果使用了注入，生成的 tokens 里不包含注入的那部分，我们需要手动拼回去显示
    input_len = inputs.input_ids.shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    if force_thinking:
        
        return "thinking[思考过程]\n" + response
    else:
        return response

# ================= 对比测试 =================

questions = [
    "计算：24.6 ÷ 0.6",
    "小明有 5 个苹果，小红的苹果数量是小明的 3 倍少 2 个。请问小红有多少个苹果？",
    "如果 3x + 5 = 20，那么 x 等于多少？"
]

separator = "=" * 80

for i, q in enumerate(questions):
    print(f"\n📝 测试题目 {i+1}: {q}")
    print(separator)
    
    # --- 原始模型 (不强制) ---
    print("🔴 [原始 0.5B 模型] 回答：")
    try:
        # 原始模型没学过这个格式，强制加反而会乱，所以不加
        ans_base = get_response(model_base, q, force_thinking=False)
        print(ans_base.strip())
    except Exception as e:
        print(e)

    print("-" * 40)
    
    # --- 蒸馏模型 (强制注入标签) ---
    print("🟢 [蒸馏后 0.5B 模型] (触发思考模式)：")
    try:
        # 我们来看看，一旦开了个头，它能不能接下去！
        ans_distilled = get_response(model_distilled, q, force_thinking=True)
        print(ans_distilled.strip())
    except Exception as e:
        print(e)
        
    print(separator)
    time.sleep(0.5)

>>> 正在初始化环境...
Loading Base Model...


The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


Loading Distilled Model...

📝 测试题目 1: 计算：24.6 ÷ 0.6
🔴 [原始 0.5B 模型] 回答：
要计算 \(24.6 \div 0.6\)，我们可以将除法转换为乘法。具体步骤如下：

\[24.6 \div 0.6 = 24.6 \times \frac{1}{0.6}\]

由于 \(0.6 = \frac{3}{5}\)，我们有：

\[24.6 \times \frac{1}{\frac{3}{5}} = 24.6 \times \frac{5}{3}\]

现在，我们将分数相乘：

\[= 8.2 \times 5\]

\[= 41\]

因此，\(24.6 \div 0.6 = 41\)。
----------------------------------------
🟢 [蒸馏后 0.5B 模型] (触发思考模式)：
thinking[思考过程]
要计算24.6除以0.6，可以将这个除法问题转化为乘法问题。具体来说，就是求解24.6乘以0.6的结果。

\[24.6 ÷ 0.6 = 24.6 × 0.6\]

我们来逐步进行计算：

1. 首先，我们可以直接相乘：
\[24.6 × 0.6 = 14.16\]

或者，也可以将24.6和0.6分别转换为分数形式，然后进行乘法运算：

\[24.6 = \frac{246}{10}，0.6 = \frac{6}{10}\]

因此，\( \frac{246}{10} × \frac{6}{10} = \frac{246×6}{10×10} = \frac{1416}{100} = 14.16 \)

所以，24.6除以0.6的最终结果是14.16。

\[24.6 ÷ 0.6 = 14.16\]

📝 测试题目 2: 小明有 5 个苹果，小红的苹果数量是小明的 3 倍少 2 个。请问小红有多少个苹果？
🔴 [原始 0.5B 模型] 回答：
小红的数量是小明的 3 倍少 2 个，所以我们可以这样计算：

小红的数量 = 小明的数量 - (小明的数量的 3 倍) + 2

将小明的数量代入公式中得到：
小红的数量 = 5 - (5 * 3) + 2
小红的数量 = 5 - 15 + 2
小红的数量 = -10 + 2
小红的数量 = -8

但是这个结果显然是不